# ValyuContext

>[Valyu](https://www.valyu.network/) allows AI applications and agents to search the internet and proprietary data sources for relevant LLM ready information.

This notebook goes over how to use Valyu context tool in LangChain.

First, get an Valyu API key and add it as an environment variable. Get $10 free credit  by [signing up here](https://exchange.valyu.network/).

## Setup

The integration lives in the `langchain-valyu` package.

In [ ]:
%pip install -qU langchain-valyu

In order to use the package, you will also need to set the `VALYU_API_KEY` environment variable to your Valyu API key.

In [ ]:
import os

valyu_api_key = os.environ["VALYU_API_KEY"]

## Instantiation

Now we can instantiate our retriever:

In [ ]:
from langchain_valyu import ValyuContextRetriever

retriever = ValyuContextRetriever(valyu_api_key=os.environ["VALYU_API_KEY"])

## Usage

In [ ]:
query = "What are the benefits of renewable energy?"
docs = retriever.invoke(query)

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)

## Use within a chain

We can easily combine this retriever in to a chain.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

llm = ChatOpenAI(model="gpt-4o-mini")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## API reference

For detailed documentation of all Valyu Context API features and configurations head to the API reference: https://docs.valyu.network/overview